In [239]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2021, 2, 1).timestamp())  # Jan 1, 2021
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/derivatives/futures_funding_rate_perpetual_v2',
    'https://api.glassnode.com/v1/metrics/derivatives/futures_open_interest_perpetual_sum',
    'https://api.glassnode.com/v1/metrics/derivatives/futures_open_interest_sum',
    'https://api.glassnode.com/v1/metrics/derivatives/futures_volume_buy_perpetual_sum',
    'https://api.glassnode.com/v1/metrics/derivatives/futures_volume_sell_perpetual_sum'
    
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'  # Default to USD
    }
    
    # Set currency to BTC for the futures_open_interest_sum metric
    if url.endswith('futures_open_interest_sum'):
        params['c'] = 'NATIVE'

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

In [240]:
# Calculate 30-day z-score of 3-day EMA of "Perpetual Long Side Interest"
def calculate_preplongsideinterest_emazscore(df, columns=['futures_funding_rate_perpetual_v2', 'futures_open_interest_perpetual_sum'], ema_window=3, norm_window=30, normalization='z-score_capped'):
    # Calculate long_interest
    df['long_interest'] = df[columns[0]] * df[columns[1]]
    
    # Calculate EMA of long_interest
    ema_long_interest = df['long_interest'].ewm(span=ema_window, adjust=False).mean()
    
    if normalization == False:
        return ema_long_interest
    elif normalization == 'maxmin':
        max_val = ema_long_interest.rolling(window=norm_window).max()
        min_val = ema_long_interest.rolling(window=norm_window).min()
        normalized_ema = 2 * (ema_long_interest - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score_capped':
        mean = ema_long_interest.rolling(window=norm_window).mean()
        std = ema_long_interest.rolling(window=norm_window).std()
        z_score = (ema_long_interest - mean) / std
        # Cap z-score between -1 and +1
        normalized_ema = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_ema, ema_long_interest

In [241]:
# Calculate 90-days z-score of 7-day change of "Future Open Interest"
def calculate_futureopeninterest_diffzscore(df, column='futures_open_interest_sum', perchange_window=7, norm_window=90, normalization='maxmin'):
    # Calculate the percent change over the specified window
    pct_change = df[column].rolling(window=perchange_window).mean()
    
    if normalization == False:
        return pct_change
    elif normalization == 'maxmin':
        max_val = pct_change.rolling(window=norm_window).max()
        min_val = pct_change.rolling(window=norm_window).min()
        normalized_pct_change = 2 * (pct_change - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score_capped':
        mean = pct_change.rolling(window=norm_window).mean()
        std = pct_change.rolling(window=norm_window).std()
        z_score = (pct_change - mean) / std
        # Cap z-score between -1.5 and +1.5, then normalize to [-1, 1]
        normalized_pct_change = z_score / z_score.abs().max()

    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score_capped'.")
    
    return normalized_pct_change

In [242]:
# Calculate 7-days sum of "Perpetual CVD"
def calculate_perp_cvd_bias(df, column=['futures_volume_buy_perpetual_sum','futures_volume_sell_perpetual_sum'], window_sum=7, norm_window=90, normalization='maxmin'):
    rolling_sum = (df[column[0]]-df[column[1]]).rolling(window=window_sum).sum()
    
    if normalization == False:
        return rolling_sum
    elif normalization == 'maxmin':
        max_val = rolling_sum.rolling(window=norm_window).max()
        min_val = rolling_sum.rolling(window=norm_window).min()
        normalized_pct_change = 2 * (rolling_sum - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score_capped':
        mean = rolling_sum.rolling(window=norm_window).mean()
        std = rolling_sum.rolling(window=norm_window).std()
        z_score = (rolling_sum - mean) / std
        # Cap z-score between -1.5 and +1.5, then normalize to [-1, 1]
        normalized_pct_change = z_score.clip(lower=-1, upper=1) /1
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score_capped'.")
    
    return normalized_pct_change, rolling_sum

In [243]:
# Set the weights for aggregation 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def aggregate_indicators(df, columns, method='equal_weight'):
    """
    Aggregate financial behavior indicators from a DataFrame, excluding initial rows with NaN values.
    
    :param df: pandas DataFrame containing the indicators
    :param columns: list of column names to be aggregated
    :param method: 'equal_weight' or 'PCA'
    :return: pandas Series with the aggregated indicator
    """
    # Ensure all specified columns exist in the DataFrame
    if not all(col in df.columns for col in columns):
        raise ValueError("Some specified columns are not in the DataFrame")
    
    # Extract the relevant columns
    data = df[columns]
    
    if method == 'equal_weight':
        # For equal weight, we can use pandas mean which automatically skips NaN
        weight = 1 / len(columns)
        print(f"weights are all equal = {weight:.4f}")
        return data.mean(axis=1)
    
    elif method == 'PCA':
        # Remove rows with any NaN values
        data_clean = data.dropna()
        
        if len(data_clean) == 0:
            raise ValueError("No complete rows found after removing NaN values")
        
        # Standardize the data
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data_clean)
        
        # Perform PCA
        pca = PCA(n_components=1)
        pca_result = pca.fit_transform(data_scaled)
        
        # Calculate weights from the first principal component
        weights = pca.components_[0] / np.sum(np.abs(pca.components_[0]))
        
        print("PCA weights:")
        for col, weight in zip(columns, weights):
            print(f"{col}: {weight:.4f}")
        
        # Calculate weighted sum for all rows, including those with NaN
        weighted_sum = data.mul(weights).sum(axis=1)
        
        # Normalize the weighted sum to be in the same range as input data
        min_val, max_val = data.min().min(), data.max().max()
        normalized_sum = (weighted_sum - weighted_sum.min()) / (weighted_sum.max() - weighted_sum.min())
        normalized_sum = normalized_sum * (max_val - min_val) + min_val
        
        return normalized_sum
    
    else:
        raise ValueError("Invalid method. Choose 'equal_weight' or 'PCA'")

In [244]:
# Apply the functions to our merged_df
merged_df['preplongsideinterest_emazscore'], merged_df['prep_long_interest_7ema'] = calculate_preplongsideinterest_emazscore(merged_df, columns=['futures_funding_rate_perpetual_v2','futures_open_interest_perpetual_sum'], normalization='z-score_capped'
)

merged_df['futureopeninterest_diffzscore'] = calculate_futureopeninterest_diffzscore(
    merged_df, column='futures_open_interest_sum', perchange_window=7, norm_window=90,normalization='maxmin'
)

merged_df['perp_cvd_bias_sum'], merged_df['perp_cvd_sum7'] = calculate_perp_cvd_bias(
    merged_df, column=['futures_volume_buy_perpetual_sum','futures_volume_sell_perpetual_sum'], window_sum =7, norm_window=90, normalization='maxmin'
)


In [245]:
# Apply the functions to our merged_df
merged_df['aggregate_indicator_equalweight']= aggregate_indicators(merged_df, columns=['preplongsideinterest_emazscore','futureopeninterest_diffzscore', 'perp_cvd_bias_sum'], method='equal_weight')
merged_df['aggregate_indicator_PCA']= aggregate_indicators(merged_df, columns=['preplongsideinterest_emazscore','futureopeninterest_diffzscore','perp_cvd_bias_sum'], method='PCA')

weights are all equal = 0.3333
PCA weights:
preplongsideinterest_emazscore: 0.4622
futureopeninterest_diffzscore: -0.0744
perp_cvd_bias_sum: 0.4634


In [246]:
# Visualize the "individual indicator"
import pandas as pd
from plotly.subplots import make_subplots

def create_indicator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=730)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator,
            name=indicator_column,
            line=dict(color='green', width=2),
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add red color for negative values
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color='red', width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 4, 7, 10]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]
    last_date = indicator.index[-1]

    # Determine the color based on the last value
    indicator_color = 'green' if last_value >= 0 else 'red'

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False,
        title_text=''  # Remove x-axis title
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

In [247]:
#visualize the 2-year "aggregated indicators"
def create_aggregator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=730)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define colors
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
    RED_COLOR = 'red'
    BLUE_COLOR = 'blue'

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    
    # Add red color for positive values with transparent fill
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator > 0, 0),
            name=f"{indicator_column} (Positive)",
            line=dict(color='green', width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0)',  # Transparent red
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add blue color for negative values with transparent fill
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color='red', width=2),
            fill='tozeroy',
            fillcolor='rgba(0,0,255,0)',  # Transparent blue
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every three months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 4, 7, 10]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]

    # Determine the color based on the last value
    indicator_color = 'green' if last_value >= 0 else 'red'

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

In [248]:
# Create charts for each indicator
indicators = [
    ('preplongsideinterest_emazscore', "Bitcoin: Perpetual Long Side Interest"),
    ('futureopeninterest_diffzscore', "Bitcoin: Futures Open Interest Change"),
    ('perp_cvd_bias_sum', "Bitcoin: Perpetual CVD Bias")
]

for indicator, title in indicators:
    fig = create_indicator_chart(merged_df, indicator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")


All charts have been displayed and saved as separate HTML files.


In [249]:
# Create charts for each aggregator
aggregators = [('aggregate_indicator_equalweight', "Indicator: Perpetual Aggregate Indicator EW"),
    ('aggregate_indicator_PCA', "Indicator: Perpetual Aggregate Indicator PCA")
    ]

for aggregator, title in aggregators:
    fig = create_aggregator_chart(merged_df, aggregator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_perp_{aggregator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [250]:
merged_df.tail()

,price_usd_close,futures_funding_rate_perpetual_v2,futures_open_interest_perpetual_sum,futures_open_interest_sum,futures_volume_buy_perpetual_sum,futures_volume_sell_perpetual_sum,long_interest,preplongsideinterest_emazscore,prep_long_interest_7ema,futureopeninterest_diffzscore,perp_cvd_bias_sum,perp_cvd_sum7,aggregate_indicator_equalweight,aggregate_indicator_PCA
t,,,,,,,,,,,,,,
2024-10-12,63185.402646,0.000053,1.786025e+10,313090.464178,1.067897e+10,1.012583e+10,9.487207e+05,0.044615,8.703310e+05,0.687247,0.328517,-8.178550e+08,0.353460,0.237834
2024-10-13,62799.208259,0.000068,1.739739e+10,307531.178655,1.299076e+10,1.313730e+10,1.185377e+06,0.146314,1.027854e+06,0.732875,0.247983,-1.320923e+09,0.375724,0.245351
2024-10-14,66058.758966,0.000101,1.872095e+10,313831.925985,3.431351e+10,3.226679e+10,1.898816e+06,0.498642,1.463335e+06,0.847273,0.617888,9.897300e+08,0.654601,0.634111
2024-10-15,67025.202753,0.000077,1.849981e+10,306440.309908,3.611608e+10,3.571320e+10,1.427133e+06,0.448219,1.445234e+06,0.898826,0.694525,1.468451e+09,0.680523,0.644100
2024-10-16,67642.151748,0.000091,1.930060e+10,315669.270309,2.700917e+10,2.627455e+10,1.761550e+06,0.499091,1.603392e+06,0.958973,0.997852,3.363211e+09,0.818639,0.834562


In [251]:
merged_df.to_csv('bitcoin_analysis_data_futures.csv', index=True)

PermissionError: [Errno 13] Permission denied: 'bitcoin_analysis_data_futures.csv'

In [ ]:
# Visualize the aggregated indicator in scatter style
def chart_aggregate(df, indicator_col, price_col, chart_title="Price and Indicator Chart"):
    # Filter for the last 2 years of data
    last_2_years = df.index.max() - pd.DateOffset(years=2)
    df_last_2_years = df[df.index >= last_2_years]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the scatter plot
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_last_2_years.index,
            y=df_last_2_years[price_col],
            mode='markers',
            marker=dict(
                color=df_last_2_years[indicator_col],
                size=8,
                line=dict(width=0.5, color='black'),  # Thinner black border for dots
                colorscale='RdYlGn_r',  # Red-Yellow-Green colorscale reversed
                colorbar=dict(
                    title='Indicator',
                    tickvals=[-1, 0, 1],
                    ticktext=['-1', '0', '1'],
                    lenmode='fraction',
                    len=0.9,  # Make the colorbar 90% of the chart height
                    thickness=20,  # Adjust thickness as needed
                    x=1.02,  # Move colorbar to the right side
                    xpad=10,  # Add some padding between colorbar and chart
                ),
                cmin=-1,
                cmax=1,
            ),
            text=[f"Date: {date.strftime('%Y-%m-%d')}<br>Price: {price:.2f}<br>Indicator: {ind:.2f}" 
                  for date, price, ind in zip(df_last_2_years.index, df_last_2_years[price_col], df_last_2_years[indicator_col])],
            hoverinfo='text'
        )
    )

    # Add vertical lines for every 3 months (January, April, July, October)
    start_date = df_last_2_years.index[0].to_period('Q').to_timestamp()
    end_date = df_last_2_years.index[-1].to_period('Q').to_timestamp()
    for date in pd.date_range(start=start_date, end=end_date, freq='Q'):
        if df_last_2_years.index[0] <= date <= df_last_2_years.index[-1]:
            fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=1, opacity=0.7)

    # Get the last value of the indicator and price
    last_value = df_last_2_years[indicator_col].iloc[-1]
    last_price = df_last_2_years[price_col].iloc[-1]

    # Add annotation for the last value
    fig.add_annotation(
        x=df_last_2_years.index[-1],  # Fixed x position at the last date
        y=last_price,  # y position based on the last price
        text=f"{last_value:.2f}",  # Simplified annotation
        showarrow=False,
        font=dict(size=14, color='red' if last_value > 0 else 'green'),  # Color logic based on reversed scale
        align="left",
        xanchor="left",
        yanchor="middle",
        xshift=10,  # Shift the annotation slightly to the right of the last point
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,
        hovermode="closest",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,
        height=450,
        margin=dict(l=40, r=120, t=60, b=40)  # Increase right margin to accommodate colorbar
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 12},
        zeroline=False,
        dtick="M3",  # Show tick every 3 months
        tickformat="%b %Y",  # Format as "Mon Year" for every tick
        showline=False,  # Remove x-axis line
        mirror=False
    )
    fig.update_yaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 12},
        zeroline=False,
        showline=True,
        linewidth=1,
        linecolor=GREY_COLOR,
        mirror=False,
        ticks='outside',  # Show ticks outside
        ticklen=5,  # Length of the ticks
        tickcolor=GREY_COLOR  # Color of the ticks
    )

    return fig

In [ ]:
# Example usage
fig2 = chart_aggregate(merged_df, 'aggregate_indicator_equalweight', 'price_usd_close', "Perpetual Sentiment Aggregated Indicator")
fig2.show()
pio.write_html(fig2, file=f'bitcoin_analysis_prep_aggr_last_2year.html')
print("All charts have been displayed and saved as separate HTML files.")

In [ ]:
# Define the color codes (matching the colors in create_indicator_chart)
GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
RED_COLOR = 'red'  # Red color for positive changes
BLUE_COLOR = 'blue'  # Blue color for negative changes

# Function to calculate percentage changes for given time periods
def calculate_periodic_changes(df, column):
    # Calculate the percentage changes for weekly, monthly, and quarterly periods
    weekly_change = df[column].pct_change(7).iloc[-1] * 100  # 7 days for weekly change
    monthly_change = df[column].pct_change(30).iloc[-1] * 100  # 30 days for monthly change
    quarterly_change = df[column].pct_change(90).iloc[-1] * 100  # 90 days for quarterly change

    return weekly_change, monthly_change, quarterly_change

In [ ]:
# Create the bar chart for quarterly, monthly, and weekly changes
def create_bar_chart(merged_df, column, chart_title="rate of change"):
    # Calculate percentage changes for the specified column
    weekly_change, monthly_change, quarterly_change = calculate_periodic_changes(merged_df, column)
    
    # Define the periods and changes in the new order: quarterly, monthly, weekly
    periods = ['Quarterly', 'Monthly', 'Weekly']
    changes = [quarterly_change, monthly_change, weekly_change]
    
    # Determine the color for each bar: GREEN_COLOR for positive, RED_COLOR for negative
    colors = ['green' if change > 0 else 'red' for change in changes]

    # Create the bar chart with filled bars and increased roundness
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=periods,
            y=changes,
            marker=dict(
                color=colors,  # Use the colors list for fill
                line=dict(
                    color=colors,  # Use the same colors for outline
                    width=2  # Adjust the line width as needed
                ),
                cornerradius=15  # Slightly reduce the roundness of corners
            ),
            text=[f"{change:.2f}%" for change in changes],  # Add text labels with percentage values
            textposition='outside',
            textfont=dict(color=GREY_COLOR, size=16),  # Set the text color to grey and adjust size
            width=0.4  # Reduce bar width to make them narrower
        )
    )

    # Customize the layout
    fig.update_layout(
        title={
            'text': f"Rate of Change {chart_title}",
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        xaxis_title=None,  # Remove the x-axis title
        yaxis_title=None,  # Remove the y-axis title
        yaxis_ticksuffix="%",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=600,  # Keep width at 600 pixels
        height=600,  # Keep height at 600 pixels
        hovermode="x unified",
        bargap=0.1  # Reduce gap between bars to bring them closer
    )

    # Add a horizontal line at y = 0%
    fig.add_hline(y=0, line_dash="solid", line_color=GREY_COLOR, line_width=1)

    # Set y-axis tickers to every 40%
    fig.update_yaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},  # Make y-axis labels grey
        zeroline=False,
        linecolor=GREY_COLOR,  # Grey color for y-axis lines
        showline=True,
        ticks='outside',
        tickcolor=GREY_COLOR,
        dtick=200  # Set dtick to 40 to display tickers every 40%
    )

    # Update x-axis (periods)
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},  # Make x-axis labels grey
        zeroline=False,
        linecolor=None,  # Remove x-axis line
        showline=False,  # Ensure no line on x-axis
        ticks='outside',
        tickcolor=GREY_COLOR
    )

    # Show the chart
    fig.show()

    # Save the chart as an HTML file
    pio.write_html(fig, file=f'{column}_PERP_rate_of_changes_chart.html')

In [ ]:
# Call the function for aggregate indicator columns and save charts
create_bar_chart(merged_df, 'aggregate_indicator_equalweight', "ROC Aggregated Sentiment (EW)")
create_bar_chart(merged_df, 'aggregate_indicator_PCA', "ROC Aggregated Sentiment (PCA)")

In [ ]:
import plotly.graph_objects as go
from datetime import datetime, timedelta
# Define the grey color as mentioned in the main code
GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

def normalize_value(value, min_val, max_val):
    range_val = max_val - min_val
    if range_val == 0:
        return 0
    normalized = (value - min_val) / range_val * 200 - 100
    return normalized

def format_number(number):
    """Format number with k, M, B suffixes"""
    abs_number = abs(number)
    if abs_number >= 1e9:
        return f"{number/1e9:.2f}B"
    elif abs_number >= 1e6:
        return f"{number/1e6:.2f}M"
    elif abs_number >= 1e3:
        return f"{number/1e3:.2f}k"
    else:
        return f"{number:.2f}"

def create_dot_plot(df, metrics_titles, days=90):
    start_date = df.index[-1] - timedelta(days=days)
    df_filtered = df[df.index >= start_date]
    
    fig = go.Figure()
    y_values = []
    
    for metric, title in metrics_titles.items():
        min_val = df_filtered[metric].min()
        max_val = df_filtered[metric].max()
        current_val = df_filtered[metric].iloc[-1]
        
        # Normalize values
        norm_min = -100
        norm_max = 100
        norm_current = normalize_value(current_val, min_val, max_val)
        
        y_values.append(title)
        
        # Add grey line
        fig.add_trace(go.Scatter(
            x=[norm_min, norm_max], y=[title, title], mode='lines',
            line=dict(color='lightgrey', width=3),
            showlegend=False
        ))
        
        # Add min value (red filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_min], y=[title], mode='markers+text',
            marker=dict(color='red', size=16, symbol='circle'),
            text=[format_number(min_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),
            name=f'{title} Min'
        ))
        
        # Add max value (green filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_max], y=[title], mode='markers+text',
            marker=dict(color='green', size=16, symbol='circle'),
            text=[format_number(max_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),
            name=f'{title} Max'
        ))
        
        # Add current value (grey halo dot)
        fig.add_trace(go.Scatter(
            x=[norm_current], y=[title], mode='markers+text',
            marker=dict(
                color='white',
                size=16,
                line=dict(color=GREY_COLOR, width=3),
                symbol='circle'
            ),
            text=[format_number(current_val)],
            textposition="top center",
            textfont=dict(color=GREY_COLOR, size=16),
            name=f'{title} Current'
        ))
    
    # Add vertical dashed lines at -50%, 0%, and 50%
    for value in [-50, 0, 50]:
        fig.add_shape(
            type="line",
            x0=value, x1=value, y0=0, y1=1,
            yref="paper",
            xref="x",
            line=dict(color=GREY_COLOR, width=1, dash="dash")
        )
    
    fig.update_layout(
        title={
            'text': f'Metric Values Over Last {days} Days',
            'font': {'color': GREY_COLOR, 'size': 20},
            'y': 0.95,  # Move the title up slightly more
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={
            'text': 'Normalized Value (%)',
            'font': {'color': GREY_COLOR, 'size': 16}
        },
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_values))),
            ticktext=y_values,
            tickfont={'color': GREY_COLOR, 'size': 16},
            side='left',
            title_standoff=35  # Add some space between axis and labels
        ),
        height=400 + (len(metrics_titles) * 30),  # Increased base height
        width=1200,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=100, r=100, t=50, b=20),  # Increased top margin
        font=dict(color=GREY_COLOR, size=14)  # Set all font colors to grey and increase base size
    )
    
    fig.update_xaxes(
        showgrid=False, 
        range=[-110, 110],
        tickvals=[-100, -50, 0, 50, 100],
        ticktext=['-100%<br>MIN-90D', '-50%', '0%', '50%', '100%<br>MAX-90D'],
        tickfont={'color': GREY_COLOR, 'size': 16},
        showline=False,
        zeroline=False
    )
    fig.update_yaxes(showgrid=False)
    
    return fig

In [ ]:
# Manually define metrics and their titles
metrics_titles = {
    'prep_long_interest_7ema': 'Perpetual Long Side Interest',
    'futures_open_interest_sum': 'Futures Open Interest',
    'perp_cvd_sum7': 'Perpetual CVD (7D-SUM)',
    # Add more metrics and titles as needed
}

# Create the dot plot
fig = create_dot_plot(merged_df, metrics_titles)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='normalized_perp_raw_metrics_dot_plot.html')